# Carregando as Bibliotecas

In [ ]:
pip install streamlit
!pip install inflection

In [ ]:
import pandas as pd
import numpy as np
# import streamlit as st
import plotly.express as px
import inflection
import plotly.graph_objs as go

# Carregando os dadops e definindo funções

In [ ]:
file_path = 'zomato.csv'
dados = pd.read_csv(file_path)

In [ ]:
#Criando um dicionário com os códigos de países

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

In [ ]:
#Criando um dicionário com os códigos de cores

COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

# Funções Auxiliares

In [ ]:
#Funçao para mostrar as dimensoes do dataframe
def show_dataframe_dimensions(dataframe):
    print(f"Number of Rows: {dataframe.shape[0]}")
    print(f"Number of Columns: {dataframe.shape[1]}")

    return None

show_dataframe_dimensions(dados)

In [ ]:
#Funçao para identificar os atributos(variaveis) numericos
def get_numerical_attributes(dataframe):
    return dataframe.select_dtypes(include=['int64', 'float64'])

get_numerical_attributes(dados)

In [ ]:
#Funcao para calcular estatídticas básicas a partir dos dados numericos
def get_first_order_statistics(dataframe):
    # Central Tendency Metrics
    mean = pd.DataFrame(dataframe.apply(np.mean)).T
    median = pd.DataFrame(dataframe.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(dataframe.apply(min)).T
    max_ = pd.DataFrame(dataframe.apply(max)).T
    range_ = pd.DataFrame(dataframe.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(dataframe.apply(np.std)).T
    skew = pd.DataFrame(dataframe.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(dataframe.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

    return m

In [ ]:
#Funçao para renomear as variaveis
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

In [ ]:
# Funçao para substituir os códigos dos países por seus nome de acordo com o dicionário COUNTRIES
def country_name(country_id):
    return COUNTRIES[country_id]

In [ ]:
#Funçao para criar categorias de preços
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [ ]:
#Funçao para reordenar as colunas
def adjust_columns_order(dataframe):
    df = dataframe.copy()
    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "price_range",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]
    return df.loc[:, new_cols_order]

In [ ]:
#Funçao para aplicar todas as funçoes criadas e  tratar os dados de uma única vez
def process_data(file_path):
    df = pd.read_csv(file_path) #Lendo o CSV

    df = df.dropna() #Removendo as linhas com NAs

    df = rename_columns(df) #Renomeando as colunas

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x)) #Criando as categorias de preços

    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x)) #Trocando códigos por nomes de países

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x)) #Trocando códigos por nomes de cores

    df["cuisines"] = df.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0]) #Dividindo a coluna de 'cuisines'

    df = df.drop_duplicates() #Removendo as duplicatas

    df = adjust_columns_order(df) #Arrumando a ordem das colunas

    df.to_csv("processed_data.csv", index=False) #Salvando de volta os dados processdos

    return df

# Limpeza de dados

##### 1. Renomear as Colunas
##### 2. FAzer uma cópia de segurança dos dados
##### 3. Identificar e remover os valores (NAN)
##### 4. Remover as linhas duplicadas
##### 5. Converter as colunas com códigos (Counry_Code, Rating_color)
##### 6. Abrir a coluna Cuisines

In [ ]:
data = process_data(file_path)

In [ ]:
#Verificando estatísticas básicas dos dadosS
get_first_order_statistics(get_numerical_attributes(data))

# Respodendo as peguntas gerais

### 1. quantos restaurantes únciso estão registrados

In [ ]:
data.restaurant_id.nunique()

### 2. Quantos países únicos estão registrados?

In [ ]:
data.country.nunique()

### 3. Quantas cidades únicas estào registradas

In [ ]:
data.city.nunique()

### 4. Qual o total de avaliações feitas?

In [ ]:
data.votes.count()

### 5. Qual o totala de tipos de culinária registrados?

In [ ]:
cuisines = data.cuisines.nunique()
cuisines

# Dos Países

## Perguntas Países

### 1. Qual o nome do país que possui mais cidades registradas?


In [ ]:
(data[['country', 'city']]
 .groupby('country')
 .nunique('city')
 .sort_values(by = 'city', ascending=False)
 .reset_index()).head(1)

### 2. Qual o nome do país que possui mais restaurantes registrados?

In [ ]:
(data[['country', 'restaurant_id']]
.groupby('country')
.nunique()
.sort_values(by = 'restaurant_id', ascending=False)
.reset_index()).head(1)

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [ ]:
(data[data['price_range'] == 4][['country', 'restaurant_id']]
 .groupby('country')
 .count()
 .sort_values(by = 'restaurant_id', ascending=False).
 reset_index()).head(1)


### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
(data[['country', 'cuisines']]
 .groupby('country')
 .nunique()
 .sort_values(by='cuisines', ascending=False)
 .reset_index()).head(1)

### 5.Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.sum()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

### 6.Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
(data[data['has_online_delivery'] == 1][['country', 'restaurant_id']]
 .groupby('country')
 .count()
 .sort_values(by='restaurant_id', ascending=False)
 .reset_index()).head(1)

### 7.Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [ ]:
(data[data['has_table_booking'] == 1][['country', 'restaurant_id']]
.groupby('country')
.nunique('restaurant_id')
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

### 8.Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.mean()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

### 9.Qual o nome do país que possui, na média, a maior nota média registrada?

In [ ]:
(data[['country', 'aggregate_rating']]
.groupby('country')
.mean()
.sort_values(by='aggregate_rating', ascending=False)
.reset_index()).head(1)

### 70.Qual o nome do país que possui, na média, a menor nota média registrada?

In [ ]:
(data[['country', 'aggregate_rating']]
.groupby('country')
.mean()
.sort_values(by='aggregate_rating', ascending=True)
.reset_index()).head(1)

### 11.Qual a média de preço de um prato para dois por país?

In [ ]:
(data[['country', 'average_cost_for_two', 'currency']]
.groupby(['country', 'currency'])
.mean()
.round(2)
.reset_index())

# Das Cidades

## Perguntas Cidades

### 1.Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
(data[['city', 'restaurant_id']]
.groupby('city')
.nunique()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

### 2.Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
(data[data['aggregate_rating'] >= 4][['city', 'restaurant_id']]
.groupby('city')
.count()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

### 3.Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [ ]:
(data[data['aggregate_rating'] <= 2.5][['city', 'restaurant_id']]
.groupby('city')
.count()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

### 4.Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
(data[['city', 'average_cost_for_two']]
.groupby('city')
.mean()
.sort_values(by='average_cost_for_two', ascending=False)
.reset_index()).head(1)

### 5.Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
(data[['city', 'cuisines']]
.groupby('city')
.nunique()
.sort_values(by='cuisines', ascending=False)
.reset_index()).head(1)

### 6.Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
(data[data['has_table_booking'] == 1][['city', 'has_table_booking']]
.groupby('city')
.sum()
.sort_values(by='has_table_booking', ascending=False).reset_index()).head(1)

### 7.Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
(data[data['is_delivering_now'] == 1][['city', 'is_delivering_now']]
.groupby('city')
.sum()
.sort_values(by='is_delivering_now', ascending=False).reset_index()).head(1)

### 8.Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
(data[data['has_online_delivery'] == 1][['city', 'has_online_delivery']]
.groupby('city')
.sum()
.sort_values(by='has_online_delivery', ascending=False).reset_index()).head(1)

# Dos Restauramtes

## Perguntas dos restaurantes

### 1.Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [ ]:
cols = [
    'restaurant_id', 'restaurant_name', 'country',
    'city', 'cuisines', 'average_cost_for_two',
    'aggregate_rating', 'votes'
]

Qual o nome do restaurante com a maior nota média?

In [ ]:
(data[cols]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]) #É possível passar uma lista para o método ascending
.reset_index()).head(1)


### 3.Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas?

In [ ]:
(data[['restaurant_name', 'restaurant_id', 'average_cost_for_two']]
.sort_values(by=['average_cost_for_two', 'restaurant_id'], ascending=[False, True])
.reset_index()).head(1)

### 4.Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Brazilian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])
.reset_index()).head(1)

### 5.Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?


In [ ]:
(data[  (data['cuisines'] == 'Brazilian') & (data['country'] == 'Brazil') ][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
.reset_index()).head(1)

### 6.Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [ ]:
(data[['has_online_delivery', 'votes']]
.groupby('has_online_delivery')
.mean()
.round(2)
.sort_values('votes', ascending=False)
.reset_index())

### 7.Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [ ]:
(data[['has_table_booking', 'average_cost_for_two']]
.groupby('has_table_booking')
.mean()
.round(2)
.sort_values('average_cost_for_two', ascending=False)
.reset_index())

### 8.Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [ ]:
japa_EUA = data[(data['cuisines'] == 'Japanese') & (data['country'] == 'United States of America')]['average_cost_for_two'].mean().round(2)
BBQ_EUA = data[(data['cuisines'] == 'BBQ') & (data['country'] == 'United States of America')]['average_cost_for_two'].mean().round(2)

print(f'O valor médio de pratos de comida japonesa para duas pessoas nos EUA é de US${japa_EUA}, enquanto o de BBQ é de US${BBQ_EUA}.')

# Dos Tipos de Culinária

## Perguntas tipos de culinária

### 1.Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
(data[data.cuisines == 'Italian'][['restaurant_name','restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
 .reset_index()).head(1)

### 2.Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Italian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)


### 3.Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
(data[data['cuisines'] == 'American'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

### 4.Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'American'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

### 5.Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Arabian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

### 6.Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Arabian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

### 7.Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?



In [ ]:
(data[data['cuisines'] == 'Japanese'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

### 8.Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?



In [ ]:
(data[data['cuisines'] == 'Japanese'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

### 9.Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?



In [ ]:
(data[data['cuisines'] == 'Home-made'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

### 10.Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?



In [ ]:
(data[data['cuisines'] == 'Home-made'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

### 11.Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?



In [ ]:
(data[['cuisines', 'average_cost_for_two']]
.groupby('cuisines')
.mean()
.round(2)
.sort_values(by='average_cost_for_two', ascending=False)
.reset_index()).head(1)

### 12.Qual o tipo de culinária que possui a maior nota média?



In [ ]:
(data[['cuisines', 'aggregate_rating']]
.groupby('cuisines')
.max()
.round(2)
.sort_values(by='aggregate_rating', ascending=False)
.reset_index()).head(20)

### 13.Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?



In [ ]:
(data[['cuisines', 'is_delivering_now']].groupby('cuisines').sum().sort_values(by='is_delivering_now', ascending=False).reset_index()).head(1)
